In [88]:
import cv2
print(cv2.__version__)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import imutils
import sys
import numpy
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pickle
numpy.set_printoptions(threshold=sys.maxsize)

%run commons.py

4.1.1


# load Naive Bayes

In [89]:
filename = 'naive_bayes_model.sav'
gnb = pickle.load(open(filename, 'rb'))

filename_blurred = 'blurred_naive_bayes_model.sav'
gnb_blurred = pickle.load(open(filename_blurred, 'rb'))

# Sector detection

In [90]:
# parameters
percentile = 80 # percentil, with rows we accept as pallets (with more black pixel than in other rows)
frame = 10 # we decice that row is important if it neighbours in frame are also in percentil
threshold = 5 # threshold of neihtbours in frame in detecting important rows
border_size = 10 # we detecting groups of important rows (mayby pallet), 
                        #this parameter is maximal space between rows

def get_pallet_sectors(img):
    hist = [255 - np.mean(row) for row in img]
    perc = np.percentile(hist, percentile)
    hist_perc = [max(x - perc, 0) for x in hist]

    def check_frame(row_id, image):
        return np.count_nonzero(image[row_id: row_id + frame]) > threshold

    detection = [ row_id for row_id, value in enumerate(hist_perc) if check_frame(row_id, hist_perc) ]

    def check_row(list_id, rows):
        if list_id == len(rows) -1 :
            return rows[list_id] - rows[list_id - 1] < border_size
        if list_id == 0:
            return rows[list_id + 1] - rows[list_id] < border_size
        return rows[list_id] - rows[list_id - 1] > border_size or rows[list_id + 1] - rows[list_id] > border_size
        
    borders = [x for x_id, x in enumerate(detection) if check_row(x_id, detection)]
    up = borders[-2] - 4
    bottom = borders[-1] + 4
    return up, bottom

## Define pipeline functions

In [91]:
def show(img):
 cv2.imshow('image',img)
 cv2.waitKey(0)
 cv2.destroyAllWindows()

In [92]:
def detect(img, model):
    w,h,ch = img.shape
    f = features(img, channels, False)
    pred = model.predict(f)
    img_pred = np.reshape(pred, (w,h,1))
    return np.logical_not(img_pred)

In [93]:
def median_filter(img):
    kernel = np.ones((5,5),np.float32)/25
    img = cv2.filter2D(img,-1,kernel)
    return cv2.threshold(img,0.5,1.0,cv2.THRESH_BINARY)[1]

In [94]:
def rotate(img, angle):
    return  1-imutils.rotate(1-img, angle)

In [95]:
def odd(num):
    return (num % 3) == 0

In [96]:
def find_pallet_by_height(img, mask_height, min_perc=0.0):
    results=[]
    mask_height = mask_height
    mask_width = int(mask_height * 5.556)
    mask_size = mask_height * mask_width
    hole_height = int(mask_height * 0.694)
    hole_width = int(mask_height * 1.58)
    hole_size = hole_height * hole_width
    hole_1_x = int(mask_height * 0.694)
    hole_1_y = int(mask_height * 0.306)
    hole_2_x = int(mask_height * 3.281)
    hole_2_y = hole_1_y
    inverse_img = np.logical_not(img)
    img_height, img_width = inverse_img.shape
    for index, x in np.ndenumerate(inverse_img):
        x,y = index
        if odd(x) and odd(y) and y+mask_height < img_height and x+mask_width < img_width:
            frame_mask = inverse_img[y:y+mask_height, x:x+mask_width]
            hole_1_y_ = y+hole_1_y
            hole_2_y_ = y+hole_2_y
            hole_1_x_ = x+hole_1_x
            hole_2_x_ = x+hole_2_x
            hole_1_mask = inverse_img[hole_1_y_:hole_1_y_+hole_height, hole_1_x_:hole_1_x_+hole_width]
            hole_2_mask = inverse_img[hole_2_y_:hole_2_y_+hole_height, hole_2_x_:hole_2_x_+hole_width]
            frame_mask_perc = np.sum(frame_mask) / mask_size
            hole_1_mask_perc = np.sum(hole_1_mask) / hole_size
            hole_2_mask_perc = np.sum(hole_2_mask) / hole_size
            perc = frame_mask_perc - hole_1_mask_perc - hole_2_mask_perc
            result=(frame_mask, perc, (x,y), mask_height)
            if perc >= min_perc:
                results.append(result)
    return results

def find_pallet(img, min_height, max_height, step, min_perc=0.0):
    results=[]
    for mask_height in range(min_height, max_height, step):
        mask_height_results = find_pallet_by_height(img, mask_height, min_perc)
        results.extend(mask_height_results)
    return results

def draw_pallets(img_full, results):
    color = (0,255,0)
    for result in results:
        (frame_mask, perc, (x,y), mask_height)=result 
        mask_width = int(mask_height * 5.556)
        img_full[y:y+mask_height, x:x+1]=color
        img_full[y:y+mask_height, x+mask_width-1:x+mask_width]=color
        img_full[y:y+1, x:x+mask_width]=color
        img_full[y+mask_height-1:y+mask_height, x:x+mask_width]=color
    return img_full

def calculate_centers(results):
    centers=[]
    for result in results:
        (frame_mask, perc, (x,y), mask_height)=result 
        mask_width = int(mask_height * 5.556)
        x = x + mask_width/2
        y = y + mask_height/2
        center = (x,y)
        centers.append(center)
    return centers

In [97]:
def save(img, file, binary=True):
    res = cv2.imwrite(file, img * 255 if binary else img)
    print("saved" if res else "save error", file)

# Set parameters

In [104]:
MIN_MASK_WIDTH = 25
MAX_MASK_WIDTH = 100
MASK_WIDTH_STEP = 3
CONF_TH = 0.28

## Run pipeline and save results

In [99]:
def denoising(img, with_save):
    img = 1-img
    kernel = np.ones((7,7), np.uint8) 
    img_erosion = cv2.erode(img, kernel, iterations=1)
    if with_save:
        save(1-img_erosion, "img_erosion.jpg")
    img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)
    if with_save:
        save(1-img_dilation, "img_dilation.jpg")
    return 1-img_dilation

def filling(img, with_save):
    img = 1-img
    kernel = np.ones((7,7), np.uint8) 
    img_dilation = cv2.dilate(img, kernel, iterations=1)
    if with_save:
        save(1-img_dilation, "img_dilation_2.jpg")
    img_erosion = cv2.erode(img_dilation, kernel, iterations=1)
    if with_save:
        save(1-img_erosion, "img_erosion_2.jpg")
    return 1-img_erosion

In [100]:
img_full = cv2.imread('r_1_11.jpg')
img_blurred = cv2.medianBlur(img_full, 7)

img_classified = detect(img_blurred, gnb_blurred).astype('float32')
save(img_classified, "img_classified.jpg")

#img_filtered = median_filter(img_classified)
#save(img_filtered, "img_filtered.jpg")
img_denoised = denoising(img_classified, True)
img_filled = filling(img_denoised, True)

img_rotated = rotate(img_filled, -2)
save(img_rotated, "img_rotated.jpg")

results = find_pallet(img_rotated, MIN_MASK_WIDTH, MAX_MASK_WIDTH, MASK_WIDTH_STEP, CONF_TH)
#save(img_pallet, "img_pallet.jpg")
print("Found pallets count:", len(results))

img_full_marked = draw_pallets(img_full, results)
save(img_full_marked, "img_full_marked.jpg", False)
centers = calculate_centers(results)
print("centers", centers)

saved img_classified.jpg
saved img_erosion.jpg
saved img_dilation.jpg
saved img_dilation_2.jpg
saved img_erosion_2.jpg
saved img_rotated.jpg
Found pallets count: 11
saved img_full_marked.jpg
centers [(494.5, 272.0), (305.0, 270.5), (305.0, 273.5), (308.0, 267.5), (308.0, 270.5), (308.0, 273.5), (308.0, 276.5), (311.0, 270.5), (311.0, 273.5), (307.0, 272.0), (310.0, 272.0)]


# Pipeline as function

In [101]:
def localise(path,filename):
    img_full = cv2.imread(path+filename)
    #img_blurred = cv2.medianBlur(img_full, 7)
    img_classified = detect(img_full, gnb).astype('float32')
    img_denoised = denoising(img_classified, False)
    img_filled = filling(img_denoised, False)
    #img_filtered = median_filter(img_classified)
    img_rotated = rotate(img_filled, -2)
    results = find_pallet(img_rotated, MIN_MASK_WIDTH, MAX_MASK_WIDTH, MASK_WIDTH_STEP, CONF_TH)
    img_full_marked = draw_pallets(img_full, results)
    save(img_full_marked, path+"localised/"+filename, False)
    return calculate_centers(results)

In [102]:
def localize_all():
    centers = dict()
    path = "/home/maciej/repos/pallet-recognition/data/jpeg_marked/"
    filenames = [f for f in sorted(listdir(path)) if isfile(join(path, f))]
    for i in tqdm(range(len(filenames))):
        filename = filenames[i]
        print(path, filename)
        centers[filename]=localise(path,filename)
    np.save(path+"eval/pred_centers", centers)

In [105]:
localize_all()



  0%|          | 0/85 [00:00<?, ?it/s]

/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_0.jpg




  1%|          | 1/85 [00:16<22:37, 16.17s/it]

saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_1_0.jpg
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_1.jpg




  2%|▏         | 2/85 [00:32<22:23, 16.18s/it]

...

saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_2_7.jpg
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_2_8.jpg




 99%|█████████▉| 84/85 [23:06<00:16, 16.52s/it]

saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_2_8.jpg
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_2_9.jpg




100%|██████████| 85/85 [23:23<00:00, 16.51s/it]

saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_2_9.jpg
